### **Introdução às LSTMs (Long Short-Term Memory)**

As redes LSTM (Long Short-Term Memory) são um tipo especial de RNN projetada para lidar com o problema de dependências de longo prazo em sequências de dados. Elas são amplamente utilizadas em tarefas de processamento de linguagem natural, reconhecimento de fala, e outras aplicações que envolvem sequências.

#### **Por que LSTMs?**

Em RNNs tradicionais, o problema de desvanecimento de gradientes pode dificultar o aprendizado de dependências de longo prazo. As LSTMs resolvem esse problema através de uma arquitetura de memória mais complexa, que inclui células de memória capazes de preservar informações ao longo de várias etapas de tempo.

#### **Arquitetura de uma LSTM**

As LSTMs introduzem três "portas" (gates) principais para controlar o fluxo de informações:
- **Porta de Entrada (Input Gate)**: Controla quanta informação da entrada atual deve ser armazenada na célula de memória.
- **Porta de Esquecimento (Forget Gate)**: Decide quanta informação da célula de memória anterior deve ser mantida.
- **Porta de Saída (Output Gate)**: Controla quanta informação da célula de memória deve ser utilizada para produzir a saída atual.

Matematicamente, a atualização de uma célula LSTM pode ser descrita pelas seguintes equações:

$$
f_t = \sigma(W_f \cdot [h_{t-1}, x_t] + b_f)
$$
$$
i_t = \sigma(W_i \cdot [h_{t-1}, x_t] + b_i)
$$
$$
\tilde{C}_t = \tanh(W_C \cdot [h_{t-1}, x_t] + b_C)
$$
$$
C_t = f_t \cdot C_{t-1} + i_t \cdot \tilde{C}_t
$$
$$
o_t = \sigma(W_o \cdot [h_{t-1}, x_t] + b_o)
$$
$$
h_t = o_t \cdot \tanh(C_t)
$$

Onde:
- $f_t$, $i_t$, $o_t$ são as portas de esquecimento, entrada e saída, respectivamente.
- $C_t$ é o estado da célula.
- $h_t$ é o estado oculto.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt

### **Primeiros Passos com LSTM - Entradas Aleatórias**

Vamos começar criando uma LSTM simples e alimentá-la com entradas aleatórias. Isso nos ajudará a entender como a LSTM processa os dados e o formato das saídas.

#### **Explicação do Código**

- **`nn.LSTM`**: Implementa uma LSTM básica com uma ou mais camadas.
- **Parâmetros principais**:
  - `input_size`: Dimensão das entradas em cada passo de tempo.
  - `hidden_size`: Número de unidades na célula LSTM.
  - `num_layers`: Número de camadas empilhadas de LSTM.
  - `batch_first=True`: Faz com que o batch seja a primeira dimensão no tensor de entrada.

In [ ]:
# Configurações
input_size = 5  # Tamanho da entrada em cada passo de tempo
hidden_size = 10  # Número de unidades na camada oculta
num_layers = 2  # Número de camadas empilhadas

# Criando uma LSTM
lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

In [ ]:
# Criando uma entrada aleatória: (batch_size, seq_len, input_size)
batch_size = 8
seq_len = 7
x = torch.randn(batch_size, seq_len, input_size)

# Executando a LSTM
output, (hn, cn) = lstm(x)

### **Interpretação das Saídas da LSTM**

Vamos analisar o que as diferentes saídas da LSTM significam:

- **`output`**: Contém a saída de cada passo de tempo para cada sequência no batch. A forma será `(batch_size, seq_len, hidden_size)`.
- **`hn`**: O estado oculto final para cada camada e cada sequência no batch. A forma será `(num_layers, batch_size, hidden_size)`.
- **`cn`**: O estado da célula final para cada camada e cada sequência no batch. A forma será a mesma que `hn`.

In [ ]:
# Analisando as saídas
print("Shape da saída: ", output.shape)
print("Shape do estado oculto hn: ", hn.shape)
print("Shape do estado da célula cn: ", cn.shape)

In [ ]:
# Modificando o comprimento da sequência
seq_len = 5
x = torch.randn(batch_size, seq_len, input_size)
output, (hn, cn) = lstm(x)

print("Novo comprimento da sequência:", seq_len)
print("Shape da saída: ", output.shape)
print("Shape do estado oculto hn: ", hn.shape)
print("Shape do estado da célula cn: ", cn.shape)

# Modificando o tamanho do batch
batch_size = 32
x = torch.randn(batch_size, seq_len, input_size)
output, (hn, cn) = lstm(x)

print("\nNovo tamanho do batch:", batch_size)
print("Shape da saída: ", output.shape)
print("Shape do estado oculto hn: ", hn.shape)
print("Shape do estado da célula cn: ", cn.shape)

### **Usando um Estado Inicial de LSTM**

Normalmente, as LSTMs começam com um estado oculto e um estado de célula inicializados como zeros. No entanto, podemos fornecer estados iniciais personalizados.

In [ ]:
# Estado inicial personalizado
h0 = torch.randn(num_layers, batch_size, hidden_size)
c0 = torch.randn(num_layers, batch_size, hidden_size)

# Executando a LSTM com estados iniciais personalizados
output, (hn, cn) = lstm(x, (h0, c0))

print("Shape da saída: ", output.shape)
print("Shape do estado oculto hn: ", hn.shape)
print("Shape do estado da célula cn: ", cn.shape)

### **Classificação de Nacionalidade de Nomes com LSTM**

Neste exemplo, vamos usar uma LSTM para classificar a nacionalidade de nomes de pessoas. O conjunto de dados contém nomes associados às suas respectivas nacionalidades. A tarefa da LSTM será aprender a classificar corretamente esses nomes em diferentes nacionalidades.

#### **Descrição do Problema**

Vamos utilizar um conjunto de dados que contém nomes e suas respectivas nacionalidades. A LSTM será treinada para prever a nacionalidade com base no nome dado.

In [ ]:
# Download do conjunto de dados em data/
!wget -P data/ https://download.pytorch.org/tutorial/data.zip

# Descompactando o arquivo
!unzip data/data.zip -d .

In [ ]:
import os
import glob
import unicodedata
import string
from tqdm import tqdm

In [ ]:
class NameDataset(Dataset):
    def __init__(self, data_path):
        self.all_letters = "-" + string.ascii_letters + " .,;'"
        self.n_letters = len(self.all_letters)
        self.NULL_IDX = 0
        self.cat2idx, self.idx2cat, self.data = self.load_data(data_path)
        self.letter2idx = {letter: idx for idx, letter in enumerate(self.all_letters)}

    # Carregar e processar os dados
    def load_data(self, path):
        cat2idx = {}
        idx2cat = {}
        data = []
        for idx, filename in enumerate(glob.glob(path)):
            category = os.path.splitext(os.path.basename(filename))[0]
            cat2idx[category] = idx
            idx2cat[idx] = category

            for line in open(filename, encoding='utf-8').read().strip().split('\n'):
                name = self.unicode_to_ascii(line)
                data.append((name, category))
        return cat2idx, idx2cat, data
    
    # Função auxiliar para remover acentos
    def unicode_to_ascii(self, s):
        return ''.join(
            c for c in unicodedata.normalize('NFD', s)
            if unicodedata.category(c) != 'Mn'
        )

    # Converter letra para tensor <1 x n_letters>
    def letter_to_tensor(self, letter):
        tensor = torch.zeros(1, self.n_letters)
        tensor[0][self.letter2idx.get(letter, self.NULL_IDX)] = 1
        return tensor

    # Converter nome para tensor <name_length x 1 x n_letters>
    def name_to_tensor(self, name):
        tensor = torch.zeros(len(name), 1, self.n_letters)
        for li, letter in enumerate(name):
            tensor[li][0][self.letter2idx.get(letter, self.NULL_IDX)] = 1
        return tensor

    def tensor_to_name(self, tensor):
        idx = torch.argmax(tensor, dim=-1)
        return ''.join(self.all_letters[i] for i in idx)

    # Retornar o tamanho do dataset
    def __len__(self):
        return len(self.data)

    # Recuperar um item do dataset
    def __getitem__(self, idx):
        name, category = self.data[idx]
        name = self.name_to_tensor(name)
        category = torch.tensor([self.cat2idx[category]], dtype=torch.long)
        return name, category

# Exemplo de uso do dataset
dataset = NameDataset('data/names/*.txt')

# Exibir o tamanho do dataset e um exemplo de tensor
print(f"Dataset size: {len(dataset)}")
name, category = dataset[0]
print(name.shape, category, dataset.tensor_to_name(name), dataset.idx2cat[category.item()])

In [ ]:
# Caminho para os arquivos de dados
data_path = 'data/names/*.txt'

# Criação do dataset
dataset = NameDataset(data_path)

# Divisão do dataset em treinamento e teste
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

print(f"Tamanho do dataset de treinamento: {len(train_dataset)}")
print(f"Tamanho do dataset de teste: {len(test_dataset)}")

In [ ]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    names, categories = zip(*batch)

    # Padding dos tensores de nomes usando o valor do caractere nulo
    names_padded = pad_sequence(names, batch_first=True, padding_value=dataset.NULL_IDX).squeeze()

    # Converte lista de categorias para tensor
    categories = torch.cat(categories)

    return names_padded, categories


# Criação dos DataLoaders
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

In [ ]:
# Exemplo de uso do DataLoader
names, categories = next(iter(test_loader))
print(names.shape, categories.shape)

In [ ]:
idx = 2
dataset.tensor_to_name(names[idx]), dataset.idx2cat[categories[idx].item()]

### **Treinamento do Modelo LSTM para Classificação de Nacionalidade**

Vamos criar e treinar uma LSTM para classificar os nomes em diferentes nacionalidades.

#### **Configuração do Modelo**

- **`input_size`**: Número de letras possíveis no nome (dimensão do vetor one-hot para cada letra).
- **`hidden_size`**: Número de unidades na camada oculta da LSTM.
- **`output_size`**: Número de categorias (nacionalidades).
- **`num_layers`**: Número de camadas LSTM empilhadas.

In [ ]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, h=None):
        # x: (batch_size, seq_len, input_size)
        batch_size, seq_len, _ = x.size()

        # h: (num_layers, batch_size, hidden_size)
        if h is None:
            h = (torch.zeros(self.lstm.num_layers, batch_size, self.lstm.hidden_size).to(x.device),
                 torch.zeros(self.lstm.num_layers, batch_size, self.lstm.hidden_size).to(x.device))

        # Processa a sequência com a LSTM
        out, (hn, cn) = self.lstm(x, h)
        
        # Apenas a última saída de tempo é usada para classificação
        out = out[:, -1, :]
        
        # Calcula a saída
        y = self.fc(out)
        return y

In [ ]:
# Configuração do modelo
input_size = dataset.n_letters
hidden_size = 128
output_size = len(dataset.cat2idx)  # Número de nacionalidades
num_layers = 1

model = LSTMClassifier(input_size, hidden_size, output_size, num_layers)

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Treinamento
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    for names, categories in tqdm(train_loader):
        optimizer.zero_grad()
        outputs = model(names)
        loss = criterion(outputs, categories.squeeze())
        loss.backward()
        optimizer.step()
    
    if epoch % 1 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

### **Avaliação do Modelo**

Após o treinamento, vamos avaliar o modelo no conjunto de teste para verificar como ele se sai na classificação das nacionalidades.

In [ ]:
# Avaliação do modelo
model.eval()
correct = 0

with torch.no_grad():
    for names, categories in test_loader:
        outputs = model(names)
        predicted = torch.argmax(outputs, dim=1)
        correct += (predicted == categories.squeeze()).sum().item()

accuracy = correct / len(test_dataset)
print(f'Acurácia no conjunto de teste: {accuracy * 100:.2f}%')

### **Visualização de Resultados**

Finalmente, podemos visualizar algumas das classificações feitas pelo modelo para verificar como ele está tomando as decisões.

In [ ]:
for _ in range(5):
    i = np.random.randint(len(test_dataset))
    name_tensor, category = test_dataset[i]
    name_tensor = name_tensor.squeeze().unsqueeze(0)
    name = dataset.tensor_to_name(name_tensor[0])
    category = category.item()

    output = model(name_tensor)
    predicted = output.argmax(-1).item()
    print(f"Nome: {name} | Nacionalidade Real: {dataset.idx2cat[category]} | Predição: {dataset.idx2cat[predicted]}")
    print()

## Exercícios

### Exercício 1
Aumente o número de camadas para 2 e treine o modelo. O que acontece com os resultados?

### Exercício 2
Utilize o modelo treinado para fazer a predição da nacionalidade do seu nome